In [1]:
from IPython.core.display import HTML
HTML("<style>.container { width:90% !important; }</style>")

In [2]:
import shutil
import os
import filecmp
import pyfits as pf
import sys
import glob
import functions as fn
fn = reload(fn)

def SafeCopy(src,dest):
    if os.path.exists(dest):
        print 'warning, tried to overwrite %s with %s'%(dest, src)
        if filecmp.cmp(src, dest):
            print 'but it\'s OK, the files were the same anyway'
        else:
            print 'DISASTER - the files were different!\n\n\n'
    else:
        shutil.copy(src,dest)
        
def SafeMove(src,dest):
    if os.path.exists(dest):
        print 'warning, tried to overwrite %s with %s'%(dest, src)
        if filecmp.cmp(src, dest):
            print 'but it\'s OK, the files were the same anyway'
        else:
            print 'DISASTER - the files were different!\n\n\n'
    else:
        shutil.move(src,dest)      
        
def ReverseDictionary(input_dict):
    return dict((v,k) for k,v in input_dict.iteritems())

def GetFilename(filename_or_path):
    return str(filename_or_path).split('/')[-1]

def SafeCopyDir(src_dir,dest_dir, prepend_date=True, skip_eko_files=True):
    files = [_ for _ in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir,_))] #grab files, reject directories
    for fname in files:
        if skip_eko_files:
            if fname.find('eko')!=-1: continue
        source_filename = os.path.join(src_dir, fname)
        if prepend_date:
            try:
                d_file = pf.open(source_filename)
                primaryHeader = d_file[0].header
                date = primaryHeader['DATE-OBS'].split('T')[0]
                d_file.close()
            except Exception, e:
                print repr(e)
                print 'no date found for prepending in %s'%source_filename
                date = ''
                
        dest_filename = os.path.join(dest_dir, date + fname)
        if os.path.exists(dest_filename):
            print 'warning: tried to overwrite %s with %s'%(dest_filename, source_filename)
            if filecmp.cmp(source_filename, dest_filename):
                print 'but it\'s OK, the files were the same anyway'
            else:
                print '\n\n *** DISASTER - the files were different!*** \n\n\n'
        else:
            shutil.copy(source_filename,dest_filename)
            
def GetDupes(input_dict):
    import itertools
    rev_multidict = {}
    for key, value in input_dict.items():
        rev_multidict.setdefault(value, list()).append(key)
    sharing_dict = {key:values for key, values in rev_multidict.items() if len(values) > 1}
    return sharing_dict

In [3]:
source_path = '/nfs/lsst2/photocalData/data/monocam/sync/'
dest_root_path = '/nfs/lsst2/photocalData/data/monocam/sanitised7/'

In [4]:
# if not os.path.exists(dest_root_path): os.mkdir(dest_root_path)

# # create dirs to separate 1.3m and 61" datasets
# if not os.path.exists(dest_root_path + '1m3'): os.mkdir(dest_root_path + '1m3')
# if not os.path.exists(dest_root_path + '1m3/1m3'): os.mkdir(dest_root_path + '1m3/1m3')
# if not os.path.exists(dest_root_path + '1m3/calibs'): os.mkdir(dest_root_path + '1m3/calibs')
# if not os.path.exists(dest_root_path + '1m3/1m3_bad_data'): os.mkdir(dest_root_path + '1m3/1m3_bad_data')
# if not os.path.exists(dest_root_path + '1m3/1m3_metadata'): os.mkdir(dest_root_path + '1m3/1m3_metadata')
# ###
# if not os.path.exists(dest_root_path + '61inch'): os.mkdir(dest_root_path + '61inch')
# if not os.path.exists(dest_root_path + '61inch/61inch'): os.mkdir(dest_root_path + '61inch/61inch')
# if not os.path.exists(dest_root_path + '61inch/calibs'): os.mkdir(dest_root_path + '61inch/calibs')
# if not os.path.exists(dest_root_path + '61inch/61inch_bad_data'): os.mkdir(dest_root_path + '61inch/61inch_bad_data')
# if not os.path.exists(dest_root_path + '61inch/61inch_metadata'): os.mkdir(dest_root_path + '61inch/61inch_metadata')

# if not os.path.exists(dest_root_path + 'misc'): os.mkdir(dest_root_path + 'misc')
# problem_path = os.path.join(dest_root_path, '1m3/problem_files/')
# if not os.path.exists(problem_path): os.mkdir(problem_path)

    
# # Copy files from sync path
# SafeCopyDir(source_path + '20160502-154000', dest_root_path + '61inch/61inch_bad_data')
# SafeCopyDir(source_path + '20160503-114400', dest_root_path + '61inch/61inch_bad_data')
# SafeCopyDir(source_path + '20160503-184000', dest_root_path + '61inch')
# SafeCopyDir(source_path + '2016_05_04',      dest_root_path + '61inch')
# SafeCopyDir(source_path + '2016_05_05',      dest_root_path + '1m3/1m3_bad_data')
# SafeCopyDir(source_path + '2016_05_07',      dest_root_path + '1m3/1m3_bad_data')
# SafeCopyDir(source_path + '2016_05_10',      dest_root_path + '1m3/1m3')
# SafeCopyDir(source_path + '2016_05_11',      dest_root_path + '1m3/1m3')
# SafeCopyDir(source_path + '2016_05_12',      dest_root_path + '1m3/1m3')
# SafeCopyDir(source_path + '20160504',        dest_root_path + '61inch/61inch')
# SafeCopyDir(source_path + '20160505',        dest_root_path + '1m3/1m3_bad_data') #before grounding change
# SafeCopyDir(source_path + '20160506',        dest_root_path + '1m3/1m3_bad_data') #before grounding change, no back bias
# SafeCopyDir(source_path + '20160508',        dest_root_path + '1m3/1m3_bad_data') #biases & darks used to test grounding improvements
# SafeCopyDir(source_path + '20160509',        dest_root_path + '1m3/1m3') #contains CBP data in subdir 'projector'
# SafeCopyDir(source_path + '20160510',        dest_root_path + '1m3/1m3')
# SafeCopyDir(source_path + '20160511',        dest_root_path + '1m3/1m3')

In [5]:
dirlist = ['61inch/61inch','61inch/61inch_bad_data','1m3/1m3','1m3/1m3_bad_data']
fullpath_dirlist = [os.path.join(dest_root_path, dirname) for dirname in dirlist]

md_files = []
for path in fullpath_dirlist: # gather all the shutter metadata files - they all have g16 in their name
    for md_file in glob.glob(path + '/*g16*'):
        md_files.append(md_file)

for md_file in md_files:
    if md_file.find('.eko')!=-1:
        print 'Found mystery file %s'%md_file
        dest_filename = dest_root_path + 'misc/' + GetFilename(md_file)
#         print 'would have moved %s to %s'%(md_file, dest_filename)
        SafeMove(md_file, dest_filename)
        continue

    f = pf.open(md_file)
    primaryHeader = f[0].header
    origin = primaryHeader['ORIGIN']
    if origin!='USNO-FS': #we found a non-shutter-metadata file! glob must have gone wrong :/
        print 'Skipping non-shutter file %s'%filename
        f.close()
        continue

    telescope  = primaryHeader['TELESCOP']
    f.close()
    
    if telescope == 'NO13':
        dest_filename = os.path.join(dest_root_path, '1m3/1m3_metadata', GetFilename(md_file))
#         print 'would have moved %s to %s'%(md_file, dest_filename)
        SafeMove(md_file, dest_filename)
    elif telescope == 'NO61':
        dest_filename = os.path.join(dest_root_path, '61inch/61inch_metadata', GetFilename(md_file))
#         print 'would have moved %s to %s'%(md_file, dest_filename)
        SafeMove(md_file, dest_filename)
    else:
        print 'Found mystery file %s'%filename
        dest_filename = os.path.join(dest_root_path, 'misc/', GetFilename(md_file))
#         print 'would have moved %s to %s'%(md_file, dest_filename)
        SafeMove(md_file, dest_filename)

In [6]:
calibs_1m3 = []
calibs_61inch = []

for fname in os.listdir(os.path.join(dest_root_path,'1m3/1m3/')):
    filename = os.path.join(dest_root_path,'1m3/1m3/', fname)
    if fname.find('bias')!=-1 or filename.find('flat')!=-1 or filename.find('dark')!=-1:
        calibs_1m3.append(filename)
        
for fname in os.listdir(os.path.join(dest_root_path,'61inch/61inch/')):
    filename = os.path.join(dest_root_path,'61inch/61inch/', fname)
    if fname.find('bias')!=-1 or filename.find('flat')!=-1 or filename.find('dark')!=-1:
        calibs_61inch.append(filename)

for filename in calibs_1m3:
    SafeMove(filename, os.path.join(dest_root_path,'1m3/calibs/',GetFilename(filename)))

for filename in calibs_61inch:
    SafeMove(filename, os.path.join(dest_root_path,'61inch/calibs/',GetFilename(filename)))

In [7]:
pairings_real_data = {}

data_path = os.path.join(dest_root_path,'1m3/1m3/')
md_path = os.path.join(dest_root_path,'1m3/1m3_metadata/')

n_problem_files = 0
for i, fname in enumerate(os.listdir(data_path)):
    filename = os.path.join(data_path, fname)
    try:
        f = pf.open(filename)
        primaryHeader = f[0].header
        header_number = primaryHeader['NUMBER']
        date = primaryHeader['DATE-OBS'].split('T')[0]
        time = primaryHeader['DATE-OBS'].split('T')[1]
        f.close()
    except Exception, e:
        print repr(e)
        print 'error in %s'%filename
        n_problem_files += 1
        continue #need to skip this file otherwise it will pick up an entry with the value from last iteration
    
    glob_string = md_path + date + 'g*.' + str(header_number).zfill(3)
    mdfiles = glob.glob(glob_string)
    if len(mdfiles) != 1:
        print 'Found %s md files for %s'%(len(mdfiles),filename)
#         problem_dest = data_path + 'no_matching_md/' + fname
#         SafeMove(filename, problem_dest)
    else:
        pairings_real_data[filename] = mdfiles[0]

print 'Scanned %s files, found %s matches with %s problem files'%(len(os.listdir(data_path)),
                                                                  len(pairings_real_data),
                                                                  n_problem_files)

Scanned 995 files, found 995 matches with 0 problem files


In [8]:
n_datafiles = len(pairings_real_data)
n_unique_mdfiles = len(ReverseDictionary(pairings_real_data))
print '%s datafiles with matches'%n_datafiles
print '%s of these datafiles are unique'%n_unique_mdfiles

data_shares = GetDupes(pairings_real_data)

for key, value in data_shares.items():
    print '\nShutter file %s is shared by datafiles:'%key
    for filename in value:
        print filename

print
#move these to the problem dir:
for key, value in data_shares.items():
    print 'Moving shutter file %s and its corresponding datafiles to problem dir:'%key
    SafeMove(key, os.path.join(problem_path,GetFilename(key)))
    for filename in value:
        SafeMove(filename, os.path.join(problem_path,GetFilename(filename)))
        pairings_real_data.pop(filename)
    data_shares.pop(key)

995 datafiles with matches
995 of these datafiles are unique



In [9]:
pairings_calib_data = {}
orphaned_calibs = []

data_path = os.path.join(dest_root_path,'1m3/calibs/')
md_path = os.path.join(dest_root_path,'1m3/1m3_metadata/')

n_problem_files = 0
for i, fname in enumerate(os.listdir(data_path)):
    filename = os.path.join(data_path, fname)
    
    # skip these files as they *will* collide, and shouldn't ever be claiming shutter files
    # But! We need to make dummy shutter files for these for the ingest phase, so we need to keep track of them!
    if filename.find('bias')!=-1 or filename.find('dark')!=-1:
        orphaned_calibs.append(filename)
        continue
        
    try:
        f = pf.open(filename)
        primaryHeader = f[0].header
        header_number = primaryHeader['NUMBER']
        date = primaryHeader['DATE-OBS'].split('T')[0]
        time = primaryHeader['DATE-OBS'].split('T')[1]
        f.close()
    except Exception, e:
        print repr(e)
        print 'error in %s'%filename
        n_problem_files += 1
        continue #need to skip this file otherwise it will pick up an entry with the value from last iteration
    
    glob_string = md_path + date + 'g*.' + str(header_number).zfill(3)
    mdfiles = glob.glob(glob_string)
    if len(mdfiles) != 1:
        print 'Found %s md files for %s'%(len(mdfiles),filename)
#         problem_dest = data_path + 'no_matching_md/' + fname
#         SafeMove(filename, problem_dest)
    else:
        pairings_calib_data[filename] = mdfiles[0]

print 'Scanned %s files, found %s matches with %s problem files'%(len(os.listdir(data_path)),
                                                                  len(pairings_calib_data),
                                                                  n_problem_files)

print 'Found %s calibs which will need fake shutter files creating for them'%len(orphaned_calibs)

Scanned 887 files, found 156 matches with 0 problem files
Found 731 calibs which will need fake shutter files creating for them


In [10]:
n_datafiles = len(pairings_calib_data)
n_unique_mdfiles = len(ReverseDictionary(pairings_calib_data))
print '%s datafiles with matches'%n_datafiles
print '%s of these datafiles are unique'%n_unique_mdfiles

calib_shares = GetDupes(pairings_calib_data)

for md_file, datafiles in calib_shares.items():
    print '\nShutter file %s is shared by datafiles:'%md_file
    for data_filename in datafiles:
        print data_filename
        
#This seems fine, so no need to pop anything

156 datafiles with matches
156 of these datafiles are unique


In [11]:
print 'number of data pairings = %s'%len(pairings_real_data)
print 'number of calib pairings = %s'%len(pairings_calib_data)

all_pairings = pairings_real_data.copy()

for k,v in pairings_calib_data.items():
    if k in all_pairings: print 'DISASTER'
    all_pairings[k] = v

print 'number of combined pairings = %s'%len(all_pairings)
n_unique_mdfiles_all_pairings = len(ReverseDictionary(all_pairings))
print 'number of combined pairings reversed= %s'%n_unique_mdfiles_all_pairings

all_shares = GetDupes(all_pairings)

for md_file, datafiles in all_shares.items():
    print '\nShutter file %s is shared by datafiles:'%md_file
    for data_filename in datafiles:
        print data_filename
        
#move these to the problem dir:
for key, value in all_shares.items():
    print 'Moving shutter file %s and its corresponding datafiles to problem dir:'%key
    SafeMove(key, os.path.join(problem_path,GetFilename(key)))
    for filename in value:
        SafeMove(filename, os.path.join(problem_path,GetFilename(filename)))
        try:
            pairings_real_data.pop(filename)
        except:
            pass
        try:
            pairings_calib_data.pop(filename)
        except:
            pass


    all_shares.pop(key)

number of data pairings = 995
number of calib pairings = 156
number of combined pairings = 1151
number of combined pairings reversed= 1151


In [12]:
# don't know why I'm opening this file twice, but I can't be bothered to change it now
n_flats_fixed = 0
for calib_filename, shutter_filename in pairings_calib_data.items():
    obj = ''
    filt = ''
    
    if calib_filename.find('flat')!=-1: # fix filter given a date range
        n_flats_fixed += 1
        obj = 'FLAT'
        shutter_file = pf.open(shutter_filename)# NB Need the filter from the _shutter_ file here
        shutterHeader = shutter_file[0].header
        filt = shutterHeader['FILTER']
        date = shutterHeader['DATE-OBS']
        time = shutterHeader['UTC']
        shutter_file.close()
        if filt == 'Y3' and date=='2016-05-11':
            print 'changing filter Y3 to u for %s on day %s'%(shutter_filename, date)
            filt = 'u'
        if filt == 'Y3'and date=='2016-05-10' and float(time.split(':')[0])>12:
            print 'changing filter Y3 to u from evening of %s on day %s'%(shutter_filename, date)
            filt = 'u'
        if filt == 'Y3'and date=='2016-05-10' and float(time.split(':')[0])<12:
            print 'Leaving Y3 filter from morning of 2016-05-10 unchanged'
            
    else:
        print 'ERROR: Unknown calib type found!!!'
        break
        
    shutter_file = pf.open(shutter_filename, mode='update')
    shutterHeader = shutter_file[0].header
    shutterHeader['OBJECT'] = obj
    shutterHeader['FILTER'] = filt
    shutter_file.flush()
    shutter_file.close()

print '\n Fixed %s flats (of the %s un-orphaned calubs)'%(n_flats_fixed,len(pairings_calib_data))

Leaving Y3 filter from morning of 2016-05-10 unchanged
Leaving Y3 filter from morning of 2016-05-10 unchanged
Leaving Y3 filter from morning of 2016-05-10 unchanged
Leaving Y3 filter from morning of 2016-05-10 unchanged
Leaving Y3 filter from morning of 2016-05-10 unchanged

 Fixed 156 flats (of the 156 un-orphaned calubs)


In [13]:
# Pick an arbitrary shutter file as the source of the metadata - we're going to overwrite any relevant bits
# This is naughty but I am lazy and annoyed by this whole process
original_shutter_dummy =  dest_root_path + '1m3/1m3_metadata/' + '2016-05-10g16d131.001'

n_discarded = 0
n_created = 0
for calib_filename in orphaned_calibs:
    obj = ''
    exptime = -1
    filt = ''
    
    if calib_filename.find('bias')!=-1: # just set the object type
        obj = 'BIAS'
        filt = 'OPEN'
        
        calib_file = pf.open(calib_filename)
        primaryHeader = calib_file[0].header
        exptime = primaryHeader['EXPTIME']
        date_obs = primaryHeader['DATE-OBS']
        date,time = date_obs.split('T')

        if exptime != 0.0: # THIS IS NOT COOL!
            calib_file.close()
            SafeMove(calib_filename, os.path.join(problem_path, GetFilename(calib_filename)))
            orphaned_calibs.remove(calib_filename)
            print 'Removed %s and its associated shutter file for being labelled a bias file but having an exptime of %s' %(calib_filename, exptime)
            n_discarded += 1
            continue #need to skip this one now!
        else:
            calib_file.close()

    elif calib_filename.find('dark')!=-1: # get exptime from calib file, write to shutter file
        obj = 'DARK'
        filt = 'OPEN'
        
        calib_file = pf.open(calib_filename)
        primaryHeader = calib_file[0].header
        exptime = primaryHeader['EXPTIME']
        date_obs = primaryHeader['DATE-OBS']
        date,time = date_obs.split('T')
        calib_file.close()

    else:
        print 'ERROR: Unknown calib type found!!!'
        break

    shutter_filename = dest_root_path + '1m3/1m3_metadata/' + GetFilename(calib_filename) + '_shutter_dummyfile'
    SafeCopy(original_shutter_dummy, shutter_filename)
    n_created += 1
    shutter_file = pf.open(shutter_filename, mode='update')
    shutterHeader = shutter_file[0].header
    shutterHeader['OBJECT'] = obj
    shutterHeader['EXPTIME'] = exptime
    shutterHeader['FILTER'] = filt
    shutterHeader['DATE-OBS'] = date
    shutterHeader['UTC'] = time
    shutterHeader['DATE'] = date_obs

    shutter_file.flush()
    shutter_file.close()
    pairings_calib_data[calib_filename] = shutter_filename

print '\n **** \n'
print 'Processed %s orphaned files'%len(orphaned_calibs)
print 'Discarded %s bad files (based on darks with >0s exptime)'%n_discarded
print 'Created   %s dummy shutter files'%n_created

warning, tried to overwrite /nfs/lsst2/photocalData/data/monocam/sanitised7/1m3/1m3_metadata/2016-05-11mdarks_072.fits_shutter_dummyfile with /nfs/lsst2/photocalData/data/monocam/sanitised7/1m3/1m3_metadata/2016-05-10g16d131.001
DISASTER - the files were different!



warning, tried to overwrite /nfs/lsst2/photocalData/data/monocam/sanitised7/1m3/1m3_metadata/2016-05-11mdarks_045.fits_shutter_dummyfile with /nfs/lsst2/photocalData/data/monocam/sanitised7/1m3/1m3_metadata/2016-05-10g16d131.001
DISASTER - the files were different!



warning, tried to overwrite /nfs/lsst2/photocalData/data/monocam/sanitised7/1m3/1m3_metadata/2016-05-11mdarks_068.fits_shutter_dummyfile with /nfs/lsst2/photocalData/data/monocam/sanitised7/1m3/1m3_metadata/2016-05-10g16d131.001
DISASTER - the files were different!



warning, tried to overwrite /nfs/lsst2/photocalData/data/monocam/sanitised7/1m3/1m3_metadata/2016-05-10mdark_29.fits_shutter_dummyfile with /nfs/lsst2/photocalData/data/monocam/sanitised7/1m3/1

In [14]:
# temp_filename = '/nfs/lsst2/photocalData/data/monocam/sanitised6/1m3/1m3/2016-05-10HD163466_y3_03.fits'
# fn.printHeaderData(temp_filename, max_lines=25)
# temp_md_file = pairings_real_data[temp_filename]
# print temp_md_file
# fn.printHeaderData(temp_md_file, max_lines=25)
# print '\n\n\n\ *** virgin header:'
# virgin_file = '/nfs/lsst2/photocalData/data/monocam/sync/2016_05_10/g16d131.187'
# fn.printHeaderData(virgin_file, max_lines=25)

In [15]:
datasets = [pairings_calib_data, pairings_real_data]

n_files = 0
for dataset in datasets:
    for data_filename, shutter_filename in dataset.items():
        shutter_file = pf.open(shutter_filename, mode='update')
        shutterHeader = shutter_file[0].header
        shutter_date  = shutterHeader['DATE-OBS']
        shutter_UTC   = shutterHeader['UTC']
        filt          = shutterHeader['FILTER']
        
        if filt == 'Y3' and shutter_date=='2016-05-11':
            print 'changing filter Y3 to u for %s on day %s'%(shutter_filename, shutter_date)
            filt = 'u'
        if filt == 'Y3'and shutter_date=='2016-05-10' and float(shutter_UTC.split(':')[0])>12:
            print 'changing filter Y3 to u from evening of %s on day %s'%(shutter_filename, shutter_date)
            filt = 'u'
        if filt == 'Y3'and shutter_date=='2016-05-10' and float(shutter_UTC.split(':')[0])<12:
            print 'Leaving Y3 filter from morning of 2016-05-10 unchanged. Corresponding files:'
            print '%s, %s'%(data_filename, shutter_filename)
        
        shutterHeader['FILTER'] = filt
        shutter_file.flush()
        shutter_file.close()

        new_date_obs = shutter_date + 'T' + shutter_UTC

        data_file = pf.open(data_filename, mode='update')
        primaryHeader = data_file[0].header
        primaryHeader['DATE-OBS'] = new_date_obs
        data_file.flush()
        data_file.close()
        n_files += 1
        if n_files %100 ==0:
            print 'Fixed %s files'%n_files
            sys.stdout.flush()

            
print 'Finished fixing %s files'%n_files

Leaving Y3 filter from morning of 2016-05-10 unchanged. Corresponding files:
/nfs/lsst2/photocalData/data/monocam/sanitised7/1m3/calibs/2016-05-10skyflats_09.fits, /nfs/lsst2/photocalData/data/monocam/sanitised7/1m3/1m3_metadata/2016-05-10g16d131.009
Fixed 100 files
Leaving Y3 filter from morning of 2016-05-10 unchanged. Corresponding files:
/nfs/lsst2/photocalData/data/monocam/sanitised7/1m3/calibs/2016-05-10skyflats_07.fits, /nfs/lsst2/photocalData/data/monocam/sanitised7/1m3/1m3_metadata/2016-05-10g16d131.007
Fixed 200 files
Leaving Y3 filter from morning of 2016-05-10 unchanged. Corresponding files:
/nfs/lsst2/photocalData/data/monocam/sanitised7/1m3/calibs/2016-05-10skyflats_06.fits, /nfs/lsst2/photocalData/data/monocam/sanitised7/1m3/1m3_metadata/2016-05-10g16d131.006
Fixed 300 files
Leaving Y3 filter from morning of 2016-05-10 unchanged. Corresponding files:
/nfs/lsst2/photocalData/data/monocam/sanitised7/1m3/calibs/2016-05-10skyflats_10.fits, /nfs/lsst2/photocalData/data/monoca

In [16]:
filters_on_sky = {}
filters_flats = {}
n_biases = 0
n_darks = 0

# Count the calibs
for data_filename, shutter_filename in pairings_calib_data.items():
    if data_filename.find('bias')!=-1:
        n_biases+=1
        continue
    if data_filename.find('dark')!=-1:
        n_darks+=1
        continue

    shutter_file = pf.open(shutter_filename)
    shutterHeader = shutter_file[0].header
    filt = shutterHeader['FILTER']
    if data_filename.find('flat')==-1 and filt!='OPEN':
        print 'You have a filter on a non-flat calib file!'
    if filt not in filters_flats.keys():
        filters_flats[filt] = 1
    else:
        filters_flats[filt] += 1
    shutter_file.close()
    
# Count the sky frame
for data_filename, shutter_filename in pairings_real_data.items():
    shutter_file = pf.open(shutter_filename)
    shutterHeader = shutter_file[0].header
    filt = shutterHeader['FILTER']
    if filt not in filters_on_sky.keys():
        filters_on_sky[filt] = 1
    else:
        filters_on_sky[filt] += 1
    shutter_file.close()
    
print 'Number of sky flats in fitler:'
calib_sum = 0
for key, value in filters_flats.items():
    print key +':\t'+str(value)
    calib_sum += value
print '---------- Total sky flats = %s'%calib_sum
print 'Number of darks  = %s'%n_darks
print 'Number of biases = %s'%n_biases

print '\n **** \n'
    
print 'Sky frames in filter:'
sky_sum = 0
for key, value in filters_on_sky.items():
    print key +':\t'+str(value)
    sky_sum += value
print '---------- Total sky frames = %s'%sky_sum

Number of sky flats in fitler:
SDSSR:	16
SDSSI:	17
u:	30
GRATING:	20
X:	11
Y3:	5
Z:	17
U:	3
Y4:	19
SDSSG:	18
---------- Total sky flats = 156
Number of darks  = 471
Number of biases = 260

 **** 

Sky frames in filter:
SDSSR:	86
SDSSI:	69
u:	20
GRATING:	329
X:	113
Y3:	13
Z:	90
U:	30
Y4:	171
SDSSG:	74
---------- Total sky frames = 995


In [17]:
sky_objects = []
for data_filename, shutter_filename in pairings_real_data.items():
    shutter_file = pf.open(shutter_filename)
    shutterHeader = shutter_file[0].header
    obj = shutterHeader['OBJECT']
#     print obj
    if obj not in sky_objects:
        sky_objects.append(obj)
    shutter_file.close()
#     break

In [18]:
print sky_objects

['HD163466', 'BD+02_3375', 'CAblank5', 'HD158485', 'M16', 'BD+60_1753', 'HZ21', 'BD+023375', 'BD+37_2346', 'lsst171131', 'SN', 'M92', 'HD159222', 'NGC', 'HD93521', 'cosmos', 'SZ', 'M104', 'M51', '61', 'M101']


In [19]:
obj_string = ''
for obj in sky_objects:
    obj_string += obj + '^'
print obj_string

HD163466^BD+02_3375^CAblank5^HD158485^M16^BD+60_1753^HZ21^BD+023375^BD+37_2346^lsst171131^SN^M92^HD159222^NGC^HD93521^cosmos^SZ^M104^M51^61^M101^


In [20]:
obj_count = 0
for data_filename, shutter_filename in pairings_real_data.items():
    shutter_file = pf.open(shutter_filename)
    shutterHeader = shutter_file[0].header
    obj = shutterHeader['OBJECT']
#     print obj
    if obj =='CAblank5':
        obj_count += 1
    shutter_file.close()
print 'CAblank5: %s exps'%obj_count

CAblank5: 142 exps
